<a href="https://colab.research.google.com/github/Pancerro/MachineLearning/blob/master/lab5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import spacy
#Jeśli nlp obiekt został utworzony, oznacza to, że zainstalowano spaCy i pomyślnie pobrano modele i dane.
nlp = spacy.load("pl_core_news_sm")
print(nlp)
from google.colab import files

introduction_text = ('To jest po polsku' ' To tak jest czasem ze jest po polsku.')
introduction_doc = nlp(introduction_text)
#tekst jest konwertowany na obiekt, który jest rozumiany przez spaCy. Możesz użyć tej metody, aby przekonwertować dowolny tekst na przetworzony Docobiekt i wydedukować atrybuty
print([token.text for token in introduction_doc])

#Przekowertowanie pliku .text na obiekt doc
#uploaded = files.upload()
#introduction_file_doc = nlp(uploaded)
#print([token.text for token in introduction_file_doc])

#Wykrywanie zdan
about_text = ('ty Adrian Ławecki Pierwsze zdanie - jest długie a'
             ' drugie jest o wiele krótsze i niepotrzebne'
             ' trzeba je usunąć. Interesuje sie tymi zdaniami'
             ' bo są Pierwsze ciekawe.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
len(sentences)

for sentence in sentences:
     print (sentence)


#wykrywanie zdan z customowym separatorem
def set_custom_boundaries(doc):
     for token in doc[:-1]:
         if token.text == '...':
             doc[token.i+1].is_sent_start = True
     return doc

ellipsis_text = ('ty Adrian Ławecki Pierwsze zdanie - jest długie a'
             ' drugie jest o wiele krótsze i niepotrzebne'
             ' trzeba je usunąć... Interesuje... sie tymi zdaniami'
             ' bo są ciekawe...')
custom_nlp = spacy.load('pl_core_news_sm')
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
    print(sentence)

#W spaCy możesz wypisywac tokeny, wykonując iterację na Docobiekcie:
for token in about_doc:
    print (token, token.idx)
#tokeny zawieraja takze rozne atrybuty takie jak np: 
#text_with_ws drukuje tekst tokena ze spacją na końcu (jeśli występuje).
#is_alpha wykrywa, czy token składa się ze znaków alfabetu, czy nie.
#is_punct wykrywa, czy token jest symbolem interpunkcyjnym, czy nie.
#is_space wykrywa, czy żeton jest spacją, czy nie.
#shape_ wypisuje kształt słowa.
#is_stop wykrywa, czy token jest słowem stop, czy nie.
for token in about_doc:
    print(token, token.idx, token.text_with_ws,
          token.is_alpha, token.is_punct, token.is_space,
          token.shape_, token.is_stop)
#Dostosowanie proces tokenizacji, aby wykrywał tokeny na niestandardowych postaciach
from spacy.tokenizer import Tokenizer
import re
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                     suffix_search=suffix_re.search,
                     infix_finditer=infix_re.finditer,
                     token_match=None
                    )


custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(about_text)
print([token.text for token in custom_tokenizer_about_doc])

#Lista słow pomijania - slowa ktora nie maja znaczenia i mozna pominac
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)

for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)
#usuneicie slow pomijanych z tekstu
for token in about_doc:
    if not token.is_stop:
     print(token)
#Lista slow bez slow pomijanych
about_no_stopword_doc = [token for token in about_doc if not token.is_stop]
print(about_no_stopword_doc)
#uzycie lementyzacji 
conference_help_doc = nlp(about_text)
for token in conference_help_doc:
    print(token, token.lemma_)
#analiza teskotw - unikalne slowa oraz liczba slow ktore sie powtarzaja
from collections import Counter
complete_doc = nlp(about_text)
words = [token.text for token in complete_doc
         if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common(5)
print(common_words)
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print(unique_words)
#To samo ale z slowami pomijanymi
words_all = [token.text for token in complete_doc if not token.is_punct]
word_freq_all = Counter(words_all)
common_words_all = word_freq_all.most_common(5)
print(common_words_all)
#Tekst z funkcja jaka pelni w tekscie
for token in about_doc:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

#Wyodrobiona kategoria slow
nouns = []
adjectives = []
for token in about_doc:
    if token.pos_ == 'NOUN':
       nouns.append(token)
    if token.pos_ == 'ADJ':
       adjectives.append(token)

#wizualizacja analizy zależności lub nazwanych encji
#from spacy import displacy
#about_interest_text = ('Jest to interesujace do nauki' 
#                       ' Naturalny językowy proces.')
#about_interest_doc = nlp(about_interest_text)
#displacy.serve(about_interest_doc, style='dep')



#Funkcja przetwarzania wstępnego konwertuje tekst na format możliwy do analizy
#Zmniejsza tekst
#Lemmatyzuje każdy token
#Usuwa symbole interpunkcyjne
#Usuwa pomijane słowa
def is_token_allowed(token):
     if (not token or not token.string.strip() or
         token.is_stop or token.is_punct):
         return False
     return True
def preprocess_token(token):
    return token.lemma_.strip().lower()
complete_filtered_tokens = [preprocess_token(token)
                            for token in complete_doc if is_token_allowed(token)]
complete_filtered_tokens

#Za pomoca regul proba wyodrebienia imiai nazwiska za pomoca reguł
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
def extract_full_name(nlp_doc):
  pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
  matcher.add('FULL_NAME', None, pattern)
  matches = matcher(nlp_doc)
  for match_id, start, end in matches:
    span = nlp_doc[start:end]
    return span.text
print(extract_full_name(about_doc))
#szukanie numeru telefonu
matcher = Matcher(nlp.vocab)
conference_org_text = ('Zrobie na zajecia'
  ' Poszukiwanie numeru telefonicznego '
  ' "Szukaj numeru".'
  ' Zostaw swój number'
  ' numer 506 658 535')
def extract_phone_number(nlp_doc):
  pattern = [{'ORTH': '('}, {'SHAPE': 'ddd'},
             {'ORTH': ')'}, {'SHAPE': 'ddd'},
             {'ORTH': '-', 'OP': '?'},
             {'SHAPE': 'ddd'}]
  matcher.add('PHONE_NUMBER', None, pattern)
  matches = matcher(nlp_doc)
  for match_id, start, end in matches:
    span = nlp_doc[start:end]
    return span.text
conference_org_doc = nlp(conference_org_text)
print(extract_phone_number(conference_org_doc))

#Analiza tekstu szukajac zaleznosci i powiazan w tekscie
piano_text = 'Marta uczy się gry na pianinie.'
piano_doc = nlp(piano_text)
for token in piano_doc:
  print(token.text, token.tag_, token.head.text, token.dep_)

#displacy.serve(piano_doc, style='dep')
#Przegladane tekstu i zaleznosci z uzyciem drzewa
one_line_about_text = ('Adrian Ławecki czuje się dzisiaj dobrze'
  ' aktualnie studiuje w Akademi Marynarki Wojennej')
one_line_about_doc = nlp(one_line_about_text)
print([token.text for token in one_line_about_doc[2].children])
print(one_line_about_doc[5].nbor(-1))
print(one_line_about_doc[5].nbor())
print([token.text for token in one_line_about_doc[2].lefts])
print([token.text for token in one_line_about_doc[2].rights])
print(list(one_line_about_doc[5].subtree))
#zwraca ciąg, scalając zawarte w nim słowa
def flatten_tree(tree):
  return ''.join([token.text_with_ws for token in list(tree)]).strip()
print(flatten_tree(one_line_about_doc[5].subtree))


#Pomaga ci wywnioskować, o czym jest mowa w zdaniu.
conference_text = ('Jest tu organizowana konferencja'
  ' Odbywa się ona 21 Czerwca 2019 w Gdyni.')
conference_doc = nlp(conference_text)
for chunk in conference_doc.noun_chunks:
  print(chunk)

#wyodrębnienia fraz czasowników
#!pip install textacy
import textacy
about_talk_text = ('Będziemy sobie gadać o sztucznej inteligencji'
  ' i naturalnym procesie nauki'
  ' Jest fajnie')
pattern = r'(<VERB>?<ADV>*<VERB>+)'
about_talk_doc = textacy.make_spacy_doc(about_talk_text,
                                        lang='en_core_web_sm')
verb_phrases = textacy.extract.pos_regex_matches(about_talk_doc, pattern)

for chunk in verb_phrases:
  print(chunk.text)

for chunk in about_talk_doc.noun_chunks:
  print(chunk)

# wyodrębnienia nazwanych jednostek:
piano_class_text = ('Najlepsza akademia pianistyczna'
  ' Jest w takim mieście co Londyn się nazywa'
  ' jest tam klasa prowadzana przez światowych in.')
piano_class_doc = nlp(piano_class_text)
for ent in piano_class_doc.ents:
  print(ent.text, ent.start_char, ent.end_char,
        ent.label_, spacy.explain(ent.label_))
#displacy.serve(piano_class_doc, style='ent')
# użyć NER, aby wymazać nazwiska osób z tekstu
survey_text = ('Spośród 5 ankietowanych osób, James Robert,'
               ' „Julie Fuller i Benjamin Brooks lubią”  jabłka.' 
               ' Kelly Cox i Matthew Evans  „jak pomarańcze')

def replace_person_names(token):
  if token.ent_iob != 0 and token.ent_type_ == 'PERSON':
     return '[REDACTED] '
     return token.string

def redact_names(nlp_doc):
  for ent in nlp_doc.ents:
     ent.merge()
     tokens = map(replace_person_names, nlp_doc)
     return ''.join(tokens)
survey_doc = nlp(survey_text)
redact_names(survey_doc)

['To', 'jest', 'po', 'polsku', 'To', 'tak', 'jest', 'czasem', 'ze', 'jest', 'po', 'polsku', '.']
ty Adrian Ławecki Pierwsze zdanie - jest długie a drugie jest o wiele krótsze i niepotrzebne trzeba je usunąć.
Interesuje sie tymi zdaniami bo są Pierwsze ciekawe.
ty Adrian Ławecki Pierwsze zdanie - jest długie a drugie jest o wiele krótsze i niepotrzebne trzeba je usunąć...
Interesuje...
sie tymi zdaniami bo są ciekawe...
ty Adrian Ławecki Pierwsze zdanie - jest długie a drugie jest o wiele krótsze i niepotrzebne trzeba je usunąć...
Interesuje...
sie tymi zdaniami bo są ciekawe...
ty 0
Adrian 3
Ławecki 10
Pierwsze 18
zdanie 27
- 34
jest 36
długie 41
a 48
drugie 50
jest 57
o 62
wiele 64
krótsze 70
i 78
niepotrzebne 80
trzeba 93
je 100
usunąć 103
. 109
Interesuje 111
sie 122
tymi 126
zdaniami 131
bo 140
są 143
Pierwsze 146
ciekawe 155
. 162
ty 0 ty  True False False xx True
Adrian 3 Adrian  True False False Xxxxx False
Ławecki 10 Ławecki  True False False Xxxxx False
Pierwsze 18 Pierwsze  T

/usr/local/lib/python3.6/dist-packages/textacy/extract.py:338: DeprecationWarning: `pos_regex_matches()` has been deprecated! for similar but more powerful and performant functionality, use `textacy.extract.matches()` instead.
  action="once",
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:253: DeprecationWarning: [W013] As of v2.1.0, Span.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:253: DeprecationWarning: [W013] As of v2.1.0, Doc.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.


TypeError: ignored